In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

In [4]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/july_16_flood_data.parquet.gzip")

In [8]:
columns_to_drop = ['Unnamed: 0', 'diff90WaterDepth', 'diffMedWaterDepth', 'p90WaterDepth', 'p75WaterDepth', 
                   'p25WaterDepth', 'medWaterDepth', 'proportionLeqSix', 'proportionLeqFive',
                  'proportionLeqFour', 'proportionLeqThree', 'proportionLeqTwo', 'proportionLeqOne']

df.drop(columns=columns_to_drop, inplace=True)

In [10]:
df.head()

,agricultureStructureIndicator,asOfDate,policyCount,dateOfLoss,elevatedBuildingIndicator,ratedFloodZone,houseWorship,locationOfContents,numberOfFloorsInTheInsuredBuilding,nonProfitIndicator,occupancyType,originalConstructionDate,originalNBDate,amountPaidOnBuildingClaim,amountPaidOnContentsClaim,amountPaidOnIncreasedCostOfComplianceClaim,postFIRMConstructionIndicator,rateMethod,smallBusinessIndicatorBuilding,totalBuildingInsuranceCoverage,totalContentsInsuranceCoverage,yearOfLoss,primaryResidenceIndicator,buildingDamageAmount,buildingDeductibleCode,netBuildingPaymentAmount,buildingPropertyValue,causeOfDamage,condominiumCoverageTypeCode,contentsDeductibleCode,netContentsPaymentAmount,disasterAssistanceCoverageRequired,ficoNumber,floodWaterDuration,floodproofedIndicator,floodEvent,iccCoverage,netIccPaymentAmount,nfipRatedCommunityNumber,nfipCommunityName,numberOfUnits,buildingReplacementCost,replacementCostBasis,stateOwnedIndicator,waterDepth,rentalPropertyIndicator,state,reportedCity,reportedZipCode,countyCode,censusTract,censusBlockGroupFips,latitude,longitude,id
0,0,2020-01-22T16:55:53.194Z,1,1998-02-07 00:00:00+00:00,0,X,0,NaN,4.0,0,1.0,1963-01-01T00:00:00.000Z,1997-01-11T00:00:00.000Z,NaN,NaN,NaN,0,7,0,200000.0,50000.0,1998,0,382.0,0,0.00,937.0,1,N,0,0.0,0.0,612.0,0.0,0,Pineapple Express - Southern,15000.0,0.0,60294.0,"OCEANSIDE, CITY OF",1.0,937.0,A,0,0.0,0,CA,Currently Unavailable,92056.0,6073.0,6.073019e+09,6.073019e+10,33.2,-117.3,23dcb0d8-3e61-45bf-899f-b951946ce2ff
1,0,2020-01-22T16:55:53.194Z,1,2005-08-29 00:00:00+00:00,0,X,0,NaN,2.0,0,1.0,1967-07-01T00:00:00.000Z,1990-07-12T00:00:00.000Z,NaN,NaN,NaN,0,7,0,100000.0,40000.0,2005,1,NaN,0,0.00,NaN,1,N,0,0.0,0.0,654.0,0.0,0,Hurricane Katrina,30000.0,0.0,225203.0,NEW ORLEANS/ORLEANS PARISH*,1.0,NaN,A,0,0.0,0,LA,Currently Unavailable,70131.0,22071.0,2.207100e+10,2.207100e+11,29.9,-90.0,55783cdd-ccbd-4b19-930b-072def248507
2,0,2020-01-22T16:55:53.194Z,1,1998-09-28 00:00:00+00:00,0,X,0,NaN,1.0,0,1.0,1972-01-01T00:00:00.000Z,1997-07-24T00:00:00.000Z,8813.21,1720.0,0.0,0,1,0,100000.0,50000.0,1998,1,9313.0,0,8813.21,80000.0,1,N,0,1720.0,0.0,133.0,0.0,0,Hurricane Georges (Panhandle),15000.0,0.0,120274.0,SANTA ROSA COUNTY *,1.0,100000.0,A,0,0.0,0,FL,Currently Unavailable,32566.0,12113.0,1.211301e+10,1.211301e+11,30.4,-86.9,bfb5922b-1b21-4882-b1d4-b3825ff53e37
3,0,2019-09-19T13:45:58.425Z,1,1994-10-07 00:00:00+00:00,0,X,0,NaN,2.0,0,1.0,1960-01-01T00:00:00.000Z,1993-10-01T00:00:00.000Z,2906.00,0.0,0.0,0,7,0,100000.0,25000.0,1994,0,4428.0,0,2906.00,100000.0,1,N,0,0.0,0.0,NaN,0.0,0,None,NaN,0.0,450026.0,"BEAUFORT, CITY OF",1.0,0.0,A,0,0.0,0,SC,Currently Unavailable,29902.0,45013.0,4.501300e+10,4.501300e+11,32.4,-80.7,c1cf6e00-1e6d-4493-93fc-eb430ef15495
4,0,2019-09-19T13:45:58.425Z,1,1996-03-11 00:00:00+00:00,0,X,0,NaN,1.0,0,1.0,1988-01-01T00:00:00.000Z,1996-01-11T00:00:00.000Z,3875.53,1545.0,0.0,1,7,0,100000.0,25000.0,1996,1,5252.0,0,3875.53,100000.0,1,N,0,1545.0,0.0,NaN,0.0,0,None,NaN,0.0,125092.0,BREVARD COUNTY *,1.0,0.0,A,0,0.0,0,FL,Currently Unavailable,32940.0,12009.0,1.200906e+10,1.200906e+11,28.3,-80.7,cad8334c-13f2-4837-bdcf-e09591197ff7


In [11]:
# Calculate the proportion of observations with waterDepth <= n (1,2,3,4,5,6) per date and censusBlockGroupFips
df['proportionLeqOne'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips', 'latitude', 'longitude'])['waterDepth'].transform(
    lambda x: (x <= 1).mean()
)

df['proportionLeqTwo'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips', 'latitude', 'longitude'])['waterDepth'].transform(
    lambda x: (x <= 2).mean()
)

df['proportionLeqThree'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips', 'latitude', 'longitude'])['waterDepth'].transform(
    lambda x: (x <= 3).mean()
)

df['proportionLeqFour'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips', 'latitude', 'longitude'])['waterDepth'].transform(
    lambda x: (x <= 4).mean()
)

df['proportionLeqFive'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips', 'latitude', 'longitude'])['waterDepth'].transform(
    lambda x: (x <= 5).mean()
)

df['proportionLeqSix'] = df.groupby(['dateOfLoss', 'censusBlockGroupFips', 'latitude', 'longitude'])['waterDepth'].transform(
    lambda x: (x <= 6).mean()
)

In [13]:
df['dateOfLoss'] = pd.to_datetime(df['dateOfLoss'])
df['originalConstructionDate_pddatetime'] = pd.to_datetime(df['originalConstructionDate'], errors='coerce')
df['originalConstructionDate_year'] = df['originalConstructionDate_pddatetime'].dt.year

df['AgeofBuildinguntilflood'] = df['yearOfLoss'] - df['originalConstructionDate_year']

In [ ]:
# Define a function that, given a group dataframe and a date, 
# returns the median, 25th, 75th, and 90th percentile of waterDepth in the 3-day window centered on that date
def get_stats_in_window(group_df, date):
    mask = (group_df['dateOfLoss'] >= date - pd.Timedelta(days=1)) & (group_df['dateOfLoss'] <= date + pd.Timedelta(days=1))
    selected_water_depths = group_df.loc[mask, 'waterDepth']
    return {
        'medWaterDepth': selected_water_depths.median(),
        'p25WaterDepth': selected_water_depths.quantile(0.25),
        'p75WaterDepth': selected_water_depths.quantile(0.75),
        'p90WaterDepth': selected_water_depths.quantile(0.90)
    }

# Iterate over all groups and all dates, and calculate the statistics in the 3-day window
for fips, group_df in df.groupby('censusBlockGroupFips'):
    for date in group_df['dateOfLoss'].unique():  # using unique to avoid recalculating for same date
        stat = get_stats_in_window(group_df, date)
        mask = (df['censusBlockGroupFips'] == fips) & (df['dateOfLoss'] == date)
        df.loc[mask, 'medWaterDepth'] = stat['medWaterDepth']
        df.loc[mask, 'p25WaterDepth'] = stat['p25WaterDepth']
        df.loc[mask, 'p75WaterDepth'] = stat['p75WaterDepth']
        df.loc[mask, 'p90WaterDepth'] = stat['p90WaterDepth']



In [ ]:
# Define a function that, given a group dataframe and a date, 
# returns the median, 25th, 75th, and 90th percentile of waterDepth in the 3-day window centered on that date
def get_stats_in_window(group_df, date):
    mask = (group_df['dateOfLoss'] >= date - pd.Timedelta(days=1)) & (group_df['dateOfLoss'] <= date + pd.Timedelta(days=1))
    selected_water_depths = group_df.loc[mask, 'waterDepth']
    return {
        'medWaterDepth': selected_water_depths.median(),
        'p25WaterDepth': selected_water_depths.quantile(0.25),
        'p75WaterDepth': selected_water_depths.quantile(0.75),
        'p90WaterDepth': selected_water_depths.quantile(0.90)
    }

# Iterate over all groups and all dates, and calculate the statistics in the 3-day window
for fips, group_df in df.groupby('censusBlockGroupFips'):
    for date in group_df['dateOfLoss'].unique():  # using unique to avoid recalculating for same date
        stat = get_stats_in_window(group_df, date)
        mask = (df['censusBlockGroupFips'] == fips) & (df['dateOfLoss'] == date)
        df.loc[mask, 'medWaterDepth'] = stat['medWaterDepth']
        df.loc[mask, 'p25WaterDepth'] = stat['p25WaterDepth']
        df.loc[mask, 'p75WaterDepth'] = stat['p75WaterDepth']
        df.loc[mask, 'p90WaterDepth'] = stat['p90WaterDepth']



In [ ]:
#get new variable of difference between observed water depth and median and 90th quantile per 3 day window water depth grouped by census block

df['diffMedWaterDepth'] = df['waterDepth'] - df['medWaterDepth']
df['diff90WaterDepth'] = df['waterDepth'] - df['p90WaterDepth']